In [34]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import folium
from folium import Popup

In [56]:
sewage = pd.read_csv('SewageLeaksRefined.csv')
# Convert Date Columns to datetime
sewage["Noted"] = pd.to_datetime(sewage["Noted"], format = "%d/%m/%Y")
sewage["Resolved"] = pd.to_datetime(sewage["Resolved"], format = "%d/%m/%Y")

colours = ['gray','blue', 'green', 'orange', 'red', 'darkred']

In [58]:
# Real world latitudes and longitudes that cover Lobitos
lobitos_max_lat = -4.449793440238931
lobitos_min_lat = -4.460475724172084
lobitos_min_lon = -81.28962377680455
lobitos_max_lon = -81.2738937464837

# Set size of grid width and length: E.g 10x10 square, 100 sectors
grid_size = 10

# Generate center points for each Lobitos sector
lobitos_lat_center = np.linspace(lobitos_min_lat, lobitos_max_lat, grid_size)
lobitos_lon_center = np.linspace(lobitos_min_lon, lobitos_max_lon, grid_size)

# Make even lat-lon squares around each Lobitos sector center point 
lat_diffs = (lobitos_lat_center[1] - lobitos_lat_center[0])/2 # Lat difference between two center points
lobitos_lat_min = lobitos_lat_center - lat_diffs # Find Lat min by taking the Lat diff from the center point
lobitos_lat_max = lobitos_lat_center + lat_diffs # Find Lat max by adding the Lat diff to the center point

lon_diffs = (lobitos_lon_center[1] - lobitos_lon_center[0])/2
lobitos_lon_min = lobitos_lon_center - lon_diffs
lobitos_lon_max = lobitos_lon_center + lon_diffs


# Make an empty dataframe to contain all the lat/lon data required for each sector
lobitos_sectors = pd.DataFrame(columns=['sector','center_lat','center_lon','lat_min','lat_max','lon_min','lon_max'])

# Loop through grid to assign lat/lon data for each sector
n = 0
for i in range(grid_size):
    for j in range(grid_size):
        lobitos_sectors.loc[n] = [f"Sector {n}"] + [lobitos_lat_center[i]] + [lobitos_lon_center[j]] + [lobitos_lat_min[i]] + [lobitos_lat_max[i]] + [lobitos_lon_min[j]] + [lobitos_lon_max[j]]
        n += 1

In [64]:
# Assign a sewage leak to the sector its lat/lon falls within
def assign_sector(row):
    mask = (
        (row['Lat'] >= lobitos_sectors['lat_min']) &
        (row['Lat'] <= lobitos_sectors['lat_max']) &
        (row['Lon'] >= lobitos_sectors['lon_min']) &
        (row['Lon'] <= lobitos_sectors['lon_max'])
    )

    matching_sectors = lobitos_sectors[mask]
    if len(matching_sectors) > 0:
        return matching_sectors.iloc[0]['sector']
    return 'Other'

# Example of allowing for further filtering of selection e.g. by date range e.g by admin selection in web app
# startDateFilter = pd.to_datetime('01/01/2024', format ="%d/%m/%Y")
# endDateFilter = pd.to_datetime('01/05/2024', format ="%d/%m/%Y")
# sewage = sewage[
#     (sewage['Noted'] >= startDateFilter) &
#     (sewage['Noted'] <= endDateFilter)
# ]

# Example of allowing for further filtering of selection e.g. by issue type e.g by admin selection in web app
# issueTypeFilter = "Leakage"
# sewage = sewage[(sewage['IssueType'] == issueTypeFilter)]


sewage.head(50)
# Loop through sewage DF assigning a sector for each incident
sewage['coord_sector'] = sewage.apply(assign_sector, axis=1)

# Group by assigned sector and calculate metrics
sector_counts = sewage.groupby('coord_sector').agg({
    'Name': 'count',
    'Diameter': ['sum', 'mean'],  # Adding sum and mean of diameter,
    'Severity': ['sum', 'mean']   # Adding sum and mean of severity,
}).reset_index()

# Flatten column names
sector_counts.columns = ['sector', 'count', 'total diameter', 'average diameter', 'total severity', 'average severity']

# Merge with sector coordinates
lobitos_data = sector_counts.merge(
    lobitos_sectors[['sector', 'center_lat', 'center_lon']],
    how='left',
    on='sector'
)

# Filter data which falls outside the sectors
lobitos_data = lobitos_data[(lobitos_data['sector'] != 'Other')]

sewage.head(50)

,Name,Lat,Lon,Noted,Resolved,Duration,Diameter,Severity,coord_sector
0,Sewage Pool 1,-4.454175,-81.286566,2024-01-01,2024-04-01,91,5.418368,4,Sector 52
1,Sewage Pool 2,-4.460881,-81.276439,2024-04-10,2024-06-06,57,8.781285,5,Sector 8
2,Sewage Pool 3,-4.458604,-81.282343,2024-01-07,2024-10-22,289,3.528173,4,Sector 24
3,Sewage Pool 4,-4.459181,-81.279875,2024-03-02,2024-06-01,91,4.797897,2,Sector 16
4,Sewage Pool 5,-4.455458,-81.286398,2024-01-13,2024-12-28,350,7.244884,2,Sector 42
5,Sewage Pool 6,-4.459880,-81.283152,2024-04-22,2024-11-07,199,5.274620,5,Sector 14
7,Sewage Pool 8,-4.459067,-81.284829,2024-04-06,2024-06-29,84,3.531663,4,Sector 13
8,Sewage Pool 9,-4.459900,-81.284800,2024-02-21,2024-11-10,263,4.979180,5,Sector 3
9,Sewage Pool 10,-4.454926,-81.283051,2024-02-17,2024-09-22,218,7.668836,2,Sector 54
10,Sewage Pool 11,-4.457802,-81.282006,2024-02-12,2024-06-11,120,1.659976,5,Sector 24


In [66]:
def assign_colour_on_leaks(leaks):
    if leaks >= 5:
        return colours[5]
    elif leaks >= 4:
        return colours[4]
    elif leaks >= 3:
        return colours[3]
    elif leaks >= 2:
        return colours[2]
    elif leaks >= 1:
        return colours[1]
    else:
        return colours[0]

def assign_colour_on_severity(severity):
    if severity >= 5:
        return colours[5]
    elif severity >= 4:
        return colours[4]
    elif severity >= 3:
        return colours[3]
    elif severity >= 2:
        return colours[2]
    elif severity >= 1:
        return colours[1]
    else:
        return colours[0]
        
# Set Folium start point at Lobitos location
lobitosGrid = folium.Map(location=[-4.457310481797269, -81.2811891931266], zoom_start=16)

# Add the sectors as rectangles on the Folium map
for _, row in lobitos_sectors.iterrows():
    kw = {
        "color": "blue",
        "line_cap": "round",
        "fill": True,
        "fill_color": "white",
        "weight": 1,
        "tooltip": row['sector']
    }
    folium.Rectangle(
        bounds=[[row['lat_min'], row['lon_min']], [row['lat_max'], row['lon_max']]],
        line_join="round",
        dash_array="5, 5",
        **kw,
    ).add_to(lobitosGrid)


for _, row in lobitos_data.iterrows():
    # Scale the radius based on leak count (adjust multiplier as needed to give appropriate bubble sizes)
    radius = np.sqrt(row['count']) * 2

    tooltip_text = f"""
    <b>{row['sector']}</b><br>
    Number of Leaks: {row['count']}<br>
    Total Diameter: {row['total diameter']:,.2f} m<br>
    Average Diameter: {row['average diameter']:,.2f}m<br>
    Total Severity: {row['total severity']:,.2f}<br>
    Average Severity: {row['average severity']:,.2f} / 5.0
    """

    # Add circle marker
    folium.CircleMarker(
        location=[row['center_lat'], row['center_lon']],
        radius=radius,
        color=assign_colour_on_severity(row['average severity']),
        fill=True,
        fill_color=assign_colour_on_severity(row['average severity']),
        fill_opacity=0.6,
        opacity=0.6,
        tooltip=tooltip_text
    ).add_to(lobitosGrid)


lobitosGrid